In [1]:
!pip install lightgbm


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from keras.models import  Sequential
from keras.layers import Dense
import keras.activations,keras.losses
import numpy as np
from lightgbm import LGBMClassifier
import os

In [23]:
path='D:\Python\TheQuibits\Zero-shield-main\Zero-shield-main_main - Copy\ml_models\Datasets'
os.chdir(path)
lab=LabelEncoder()
data=pd.read_csv('UGRansomware.csv')
print(data.columns)
print(data.info())
print(data.isna().sum())
print(data.describe())

Index(['Time', 'Protcol', 'Flag', 'Family', 'Clusters', 'SeddAddress',
       'ExpAddress', 'BTC', 'USD', 'Netflow_Bytes', 'IPaddress', 'Threats',
       'Port', 'Prediction'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149043 entries, 0 to 149042
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Time           149043 non-null  int64 
 1   Protcol        149043 non-null  object
 2   Flag           149043 non-null  object
 3   Family         149043 non-null  object
 4   Clusters       149043 non-null  int64 
 5   SeddAddress    149043 non-null  object
 6   ExpAddress     149043 non-null  object
 7   BTC            149043 non-null  int64 
 8   USD            149043 non-null  int64 
 9   Netflow_Bytes  149043 non-null  int64 
 10  IPaddress      149043 non-null  object
 11  Threats        149043 non-null  object
 12  Port           149043 non-null  int64 
 13  Prediction     149043 non

In [24]:
new=[]
for i in data.select_dtypes(include='object').columns.values:
    if len(data[i].value_counts().values) < 10:
        new.append(i)

In [25]:
lab=LabelEncoder()
for i in data[new]:
    data[i]=lab.fit_transform(data[i])

print(data.info())


print(data['Family'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149043 entries, 0 to 149042
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Time           149043 non-null  int64 
 1   Protcol        149043 non-null  int32 
 2   Flag           149043 non-null  int32 
 3   Family         149043 non-null  object
 4   Clusters       149043 non-null  int64 
 5   SeddAddress    149043 non-null  int32 
 6   ExpAddress     149043 non-null  int32 
 7   BTC            149043 non-null  int64 
 8   USD            149043 non-null  int64 
 9   Netflow_Bytes  149043 non-null  int64 
 10  IPaddress      149043 non-null  int32 
 11  Threats        149043 non-null  int32 
 12  Port           149043 non-null  int64 
 13  Prediction     149043 non-null  int32 
dtypes: int32(7), int64(6), object(1)
memory usage: 11.9+ MB
None
Locky               25062
SamSam              19657
WannaCry            16110
JigSaw              13712
Flyper         

In [26]:
#for i in data.select_dtypes(include='number').columns.values:
    #sn.boxplot(data[i])
    #plt.show()
info={}
for i in data.select_dtypes(include='number').columns.values:
    data['z-scores']=(data[i]-data[i].mean())/data[i].std()
    outlier=np.abs(data['z-scores'] >3).sum()
    if outlier >0:
        info[i]=outlier

print(data.info())

print(len(data))
threshold=2
for x,y in info.items():
    upper=data[x].mean()+threshold*data[x].std()
    lower=data[x].mean()-threshold*data[x].std()
    data=data[(data[x]>lower)&(data[x]<upper)]

print(len(data))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149043 entries, 0 to 149042
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Time           149043 non-null  int64  
 1   Protcol        149043 non-null  int32  
 2   Flag           149043 non-null  int32  
 3   Family         149043 non-null  object 
 4   Clusters       149043 non-null  int64  
 5   SeddAddress    149043 non-null  int32  
 6   ExpAddress     149043 non-null  int32  
 7   BTC            149043 non-null  int64  
 8   USD            149043 non-null  int64  
 9   Netflow_Bytes  149043 non-null  int64  
 10  IPaddress      149043 non-null  int32  
 11  Threats        149043 non-null  int32  
 12  Port           149043 non-null  int64  
 13  Prediction     149043 non-null  int32  
 14  z-scores       149043 non-null  float64
dtypes: float64(1), int32(7), int64(6), object(1)
memory usage: 13.1+ MB
None
149043
111976


In [27]:
data['Family']=lab.fit_transform(data['Family'])


values={}
for i in data.columns.values:
    cnt=0
    corri=data.corr()[i]
    corri=corri.drop(['z-scores',i])
    for j in corri.index:
        if corri[j] <0:
            cnt +=1

    values[i]=cnt


In [28]:
x=[]
y=[]
print("The prediction for FLAG column in data set")
corri = data.corr()['Flag']
corri = corri.drop(['z-scores', 'Flag'])
x_train,x_test,y_train,y_test=train_test_split(data[corri.index],data['Flag'])

The prediction for FLAG column in data set


In [29]:
lgb = LGBMClassifier()
lgb.fit(x_train, y_train)
print(f'The LGB for column', lgb.score(x_test, y_test))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004822 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 803
[LightGBM] [Info] Number of data points in the train set: 83982, number of used features: 13
[LightGBM] [Info] Start training from score -4.403961
[LightGBM] [Info] Start training from score -1.109928
[LightGBM] [Info] Start training from score -2.764028
[LightGBM] [Info] Start training from score -3.137795
[LightGBM] [Info] Start training from score -1.993312
[LightGBM] [Info] Start training from score -2.068429
[LightGBM] [Info] Start training from score -2.350536
[LightGBM] [Info] Start training from score -2.757439
[LightGBM] [Info] Start training from score -2.037354
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

In [34]:
path='D:\Python\TheQuibits\Zero-shield-main\Zero-shield-main_main - Copy\ml_models\saved_models'
os.chdir(path)

In [35]:
import pickle
with open("LightBGM", 'wb') as file:
    pickle.dump(lgb, file)

In [36]:
file.close()